In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI

# =========================
# 1) .env 로드
# =========================
load_dotenv(override=True)

# =========================
# 2) 환경변수 읽기
# =========================
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# import pytesseract
# print(pytesseract.get_tesseract_version())

In [2]:
file = "data/210818_01 (1)_page-0004.jpg"

In [6]:
%%time
from pathlib import Path
from pypdf import PdfReader

IMAGE_EXTS = {".jpg", ".jpeg", ".png"}

def is_text_pdf(pdf_path: str, min_chars: int = 30) -> bool:
    reader = PdfReader(pdf_path)
    total_chars = 0

    for page in reader.pages:
        text = page.extract_text()
        if text:
            total_chars += len(text.strip())
        if total_chars >= min_chars:
            return True

    return False


def classify_for_ocr(path: str, min_chars: int = 30) -> dict:
    """
    분류 결과는 딱 2가지:
    - text_pdf
    - image
    """
    p = Path(path)
    ext = p.suffix.lower()   # ← '.jpg' 형태 유지

    # 1️⃣ 이미지 파일
    if ext in IMAGE_EXTS:
        return {"input_type": "image"}

    # 2️⃣ PDF 파일
    if ext == ".pdf":
        return (
            {"input_type": "text_pdf"}
            if is_text_pdf(path, min_chars)
            else {"input_type": "image"}
        )

    # 3️⃣ 그 외
    raise ValueError(f"지원하지 않는 파일 형식: {ext}")

CPU times: total: 0 ns
Wall time: 0 ns


In [7]:
%%time
file_path = file

result = classify_for_ocr(file_path)
print(result)

{'input_type': 'image'}
CPU times: total: 0 ns
Wall time: 112 μs


In [8]:
%%time
# text pdf 읽는 코드
from pypdf import PdfReader

def read_text_pdf(pdf_path: str) -> str:
    """
    텍스트 PDF를 페이지 순서대로 읽어
    하나의 문자열로 반환한다.
    """
    reader = PdfReader(pdf_path)
    pages_text = []

    for page in reader.pages:
        text = page.extract_text()
        if text:
            pages_text.append(text)

    return "\n".join(pages_text)

CPU times: total: 0 ns
Wall time: 0 ns


In [9]:
%%time
if result["input_type"] == "text_pdf":
    text = read_text_pdf(file)
    print(text[:500])
else:
    print("OCR 대상 이미지 입력")


OCR 대상 이미지 입력
CPU times: total: 0 ns
Wall time: 1.27 ms


In [10]:
%%time
# =========================
# [셀 1] OCR: EasyOCR + Tesseract 병용 (개선판)
# =========================
# 필요:
#   pip install easyocr pytesseract opencv-python pillow numpy

import re
import numpy as np
import cv2
from PIL import Image
import easyocr
import pytesseract

# (Windows에서 필요할 수 있음)
# pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# ---------- 전처리 ----------
def _read_image_gray(path: str) -> np.ndarray:
    """한글 경로까지 고려한 안전 로더 -> GRAY 반환"""
    try:
        img = cv2.imread(path)
        if img is None:
            raise ValueError("cv2.imread returned None")
        return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    except Exception:
        pil = Image.open(path).convert("RGB")
        arr = np.array(pil)
        return cv2.cvtColor(arr, cv2.COLOR_RGB2GRAY)

def upscale(gray: np.ndarray, target_min_width: int = 1800) -> np.ndarray:
    h, w = gray.shape[:2]
    if w >= target_min_width:
        return gray
    s = target_min_width / w
    return cv2.resize(gray, (int(w*s), int(h*s)), interpolation=cv2.INTER_CUBIC)

def deskew(gray: np.ndarray) -> np.ndarray:
    thr = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    coords = np.column_stack(np.where(thr > 0))
    if coords.size == 0:
        return gray
    angle = cv2.minAreaRect(coords)[-1]
    angle = -(90 + angle) if angle < -45 else -angle
    h, w = gray.shape[:2]
    M = cv2.getRotationMatrix2D((w/2, h/2), angle, 1.0)
    return cv2.warpAffine(gray, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

def denoise(gray: np.ndarray) -> np.ndarray:
    return cv2.fastNlMeansDenoising(gray, None, h=12, templateWindowSize=7, searchWindowSize=21)

def binarize(gray: np.ndarray, mode: str = "otsu") -> np.ndarray:
    if mode == "otsu":
        return cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    if mode == "adaptive":
        return cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                     cv2.THRESH_BINARY, 35, 11)
    raise ValueError("mode must be 'otsu' or 'adaptive'")

def make_variants(image_path: str) -> dict:
    """
    v1: gray -> upscale -> deskew -> otsu
    v2: gray -> upscale -> deskew -> denoise -> otsu
    v3: gray -> upscale -> deskew -> adaptive
    """
    gray = _read_image_gray(image_path)
    gray = upscale(gray, target_min_width=1800)
    gray = deskew(gray)
    return {
        "v1": binarize(gray, "otsu"),
        "v2": binarize(denoise(gray), "otsu"),
        "v3": binarize(gray, "adaptive"),
    }

# ---------- EasyOCR: 줄 재구성 ----------
_EASYOCR_READER = None

def get_easyocr_reader(langs=("ko", "en"), gpu=False):
    global _EASYOCR_READER
    if _EASYOCR_READER is None:
        _EASYOCR_READER = easyocr.Reader(list(langs), gpu=gpu)
    return _EASYOCR_READER

def stitch_lines_easyocr(results, y_tol=14) -> str:
    """
    (bbox, text, conf)들을 y로 묶어 '줄'을 만들고,
    같은 줄은 x로 정렬해 자연스럽게 합침.
    """
    items = []
    for bbox, text, conf in results:
        t = (text or "").strip()
        if not t:
            continue
        xs = [p[0] for p in bbox]
        ys = [p[1] for p in bbox]
        x_min = float(min(xs))
        y_center = float((min(ys) + max(ys)) / 2.0)
        items.append((y_center, x_min, t, float(conf)))

    if not items:
        return ""

    items.sort(key=lambda x: (x[0], x[1]))
    lines = []  # [y_ref, [(x, text), ...]]

    for y, x, t, conf in items:
        if not lines:
            lines.append([y, [(x, t)]])
            continue
        if abs(y - lines[-1][0]) <= y_tol:
            lines[-1][1].append((x, t))
        else:
            lines.append([y, [(x, t)]])

    out_lines = []
    for y_ref, parts in lines:
        parts.sort(key=lambda p: p[0])
        out_lines.append(" ".join(p[1] for p in parts).strip())

    return "\n".join(out_lines).strip()

def run_easyocr(img_gray_or_bin: np.ndarray, gpu=False) -> dict:
    reader = get_easyocr_reader(gpu=gpu)
    results = reader.readtext(img_gray_or_bin, detail=1, paragraph=False)
    confs = [float(r[2]) for r in results] if results else []
    avg_conf = float(np.mean(confs)) if confs else 0.0
    text = stitch_lines_easyocr(results, y_tol=14)
    return {"engine": "easyocr", "text": text, "avg_conf": avg_conf}

# ---------- Tesseract ----------
def run_tesseract(img_gray_or_bin: np.ndarray, psm: int = 6, oem: int = 3) -> dict:
    config = f"--oem {oem} --psm {psm}"
    text = pytesseract.image_to_string(img_gray_or_bin, lang="kor+eng", config=config) or ""
    return {"engine": "tesseract", "text": text.strip(), "psm": psm, "oem": oem}

# ---------- 후처리(최소) + 선택 ----------
def normalize_spaces(text: str) -> str:
    if not text:
        return ""
    text = re.sub(r"[ \t]+", " ", text)
    text = "\n".join(line.strip() for line in text.splitlines())
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()

def legal_cleanup_min(text: str) -> str:
    """의미 변경 없이, 자주 깨지는 표기만 최소 교정"""
    if not text:
        return ""
    reps = [
        (r"착정일자|정일자", "확정일자"),
        (r"경신|강신|껑신", "갱신"),
        (r"훨차임", "월차임"),
        (r"보종금", "보증금"),
        (r"관활", "관할"),
        (r"수 있으여", "수 있으며"),
        (r"주장할\s*잎고", "주장할 수 있고"),
        (r"변제발올", "변제받을"),
        (r"확인할\s*있습니다", "확인할 수 있습니다"),
        (r"주택임대\s*차", "주택임대차"),
        (r"주택임대차계\s*\n?\s*약", "주택임대차계약"),
    ]
    out = text
    for pat, repl in reps:
        out = re.sub(pat, repl, out)
    return out

def ocr_quality_score(text: str) -> float:
    if not text:
        return 0.0
    length = len(text)
    kor = len(re.findall(r"[가-힣]", text))
    alnum = len(re.findall(r"[A-Za-z0-9가-힣]", text))
    kor_ratio = kor / max(alnum, 1)

    tokens = re.findall(r"\S+", text)
    short_tokens = sum(1 for x in tokens if len(x) <= 2)
    frag = short_tokens / max(len(tokens), 1)

    legal_hits = 0
    for pat in [r"제\s*\d+\s*조", r"제\s*\d+\s*항", r"제\s*\d+\s*호", r"주택임대차보호법"]:
        if re.search(pat, text):
            legal_hits += 1

    score = 0.0
    score += min(length / 2000.0, 1.0) * 2.0
    score += kor_ratio * 2.0
    score += legal_hits * 0.4
    score += (1.0 - min(frag, 1.0)) * 1.5
    return float(score)

def ocr_dual(image_path: str, gpu=False) -> dict:
    variants = make_variants(image_path)
    candidates = []

    for vname, vimg in variants.items():
        e = run_easyocr(vimg, gpu=gpu)
        e_text = legal_cleanup_min(normalize_spaces(e["text"]))
        e_score = ocr_quality_score(e_text) + e["avg_conf"] * 0.8
        candidates.append({"engine": "easyocr", "variant": vname, "text": e_text, "score": e_score})

        t = run_tesseract(vimg, psm=6, oem=3)
        t_text = legal_cleanup_min(normalize_spaces(t["text"]))
        t_score = ocr_quality_score(t_text)
        candidates.append({"engine": "tesseract", "variant": vname, "text": t_text, "score": t_score})

    best = max(candidates, key=lambda x: x["score"]) if candidates else None
    return {"best": best, "candidates": sorted(candidates, key=lambda x: x["score"], reverse=True)}



CPU times: total: 2.94 s
Wall time: 3.73 s


In [11]:
%%time
# ---- 실행 예시 ----
if result["input_type"] == "image":
    path = file
    ocr_res = ocr_dual(path, gpu=False)
    print("BEST:", ocr_res["best"]["engine"], ocr_res["best"]["variant"], "score=", round(ocr_res["best"]["score"], 3))
    ocr_text = ocr_res["best"]["text"]
    print(ocr_text[:1200])
else:
    print("OCR 대상 이미지 입력")

Using CPU. Note: This module is much faster with a GPU.
C:\Users\Admin\anaconda3\envs\llm\lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


BEST: easyocr v3 score= 5.251
별지2)
계약생신 거절통지서
임대인 임차인
(성명) (성명)
(주소) (주소)
(연락처) (연락처)
임차목적물 주소
임대차계약 기간
임대인( 스J은 임차인( )로부터 스년 월 일 주택임대차계약의 갱신올 요
구받있으나, 아래와 같은 법률상 사유로 위 임차인에게 갱신요구릎 거절하다는 의사큼 통지합니다.
계약증신거절 사유(주택임대차보호법 제6조의3 제(항 각 호)
1. 임차인이 2기1의 차임액에 해당하는 금액에 이르도록 차임올 연체한 사실이 있는 경우 [
2 임차인이 거짓이나 그 밖의 부정한 방법으로 임차한 경우 다
3. 서로 합의하여 임대인이 임차인에게 상당한 보상올 제공한 경우 다
(상당한 보상의 내용
4. 임차인이 임대인의 동의 없이 목적 주택의 전부 또는 일부흘 전대( 쓸)한 경우 [
5. 임차인이 임차한 주택의 전부 또는 일부름 고의나 중대한 과실로 파손한 경우 [
6. 임차한 주택의 전부 또는 일부가 별실되어 임대차의 목적올 달성하지 못할 경우 [
7. 주택의 전부 또는 대부분올 철거: 재건축하기 위하여 점유름 회복할 필요가 짓는 경우
7-1. 임대차계약 체결 당시 공사시기 및 소요기간 등물 포함한 철거 또는 재건축 계획울 임차인에게 구체적으로
고지하고 그 계획에 따르는 경우 [
7-2. 건물이 노후 횟손 또는 일부 덜실되는 등 안전사고의 우려가 있는 경우 [
7-3. 다른 법령에 따라 철거 또는 재건축이 이루어지논 경우 다
8. 임대인 또는 임대인의 직계존비속이 목적 주택에 실제 거주하려논 경우 다
(실거주자 성명: 임대인과의 관계 다 본인 직계존속 직계비속)
9 그 밖에 임차인이 임차인으로서의 의무릎 현저히 위반하거나 임대차지 계속하기 어려운 중대한 사
유가 있는 경우 [
위 계약갱신거절 사유름 보충설명하기 위한 구체적 사정
어 7무- 가무 무_다나가 무다 무수 다가다구 구_나가 다스 수무하 무수 스_ _다수 _수_ 다스_나 가 다스 나 가 다스 나 다; 다스 _나다 나 + _가 +
관W뜰 7 ??

In [12]:
%%time
import os, re
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def llm_cleanup_text_only(raw_text: str, model="gpt-5-mini", max_output_tokens=1200) -> str:
    if not raw_text or not raw_text.strip():
        return ""

    pre = re.sub(r"[ \t]+", " ", raw_text)
    pre = "\n".join(line.strip() for line in pre.splitlines())
    pre = re.sub(r"\n{3,}", "\n\n", pre).strip()

    instructions = """
너는 한국어 계약서/법률문서 OCR 후처리 편집자다.

규칙:
- 내용 추가/삭제/요약 금지, 의미 변경 금지
- 오탈자/띄어쓰기/조사/잘못 끊긴 줄바꿈만 교정
- 조항/항/호 구조와 순서 유지
- 출력은 '정제된 본문 텍스트'만. 머리말/설명/라벨 금지.
""".strip()

    resp = client.responses.create(
        model=model,
        input=[
            {"role": "developer", "content": instructions},
            {"role": "user", "content": pre},
        ],
        # ✅ 핵심: 후처리는 추론 거의 필요 없음 → reasoning 최소화
        reasoning={"effort": "minimal"},
        # ✅ 출력은 텍스트로
        text={"format": {"type": "text"}},
        # ✅ 너무 크게 잡으면 reasoning으로 새는 경우가 있어, 적당히
        max_output_tokens=max_output_tokens,
    )

    # gpt-5 계열은 output_text가 잘 채워지는 편(이제 reasoning이 줄어서)
    out = (resp.output_text or "").strip()
    return out



CPU times: total: 46.9 ms
Wall time: 67.3 ms


In [13]:
%%time
clean = llm_cleanup_text_only(ocr_text)
print("clean length =", len(clean))
print(clean[:2000])

clean length = 1037
별지2)
계약갱신 거절통지서
임대인 임차인
(성명) (성명)
(주소) (주소)
(연락처) (연락처)
임차목적물 주소
임대차계약 기간
임대인(    )은 임차인(    )로부터    년    월    일 주택임대차계약의 갱신을 요구받았으나, 아래와 같은 법률상 사유로 위 임차인에게 갱신요구를 거절하다는 의사표시를 통지합니다.
계약갱신거절 사유(주택임대차보호법 제6조의3 제1항 각 호)
1. 임차인이 2기 이상의 차임액에 해당하는 금액에 이르도록 차임을 연체한 사실이 있는 경우
2. 임차인이 거짓이나 그 밖의 부정한 방법으로 임차한 경우
3. 서로 합의하여 임대인이 임차인에게 상당한 보상을 제공한 경우
(상당한 보상의 내용)
4. 임차인이 임대인의 동의 없이 목적 주택의 전부 또는 일부를 전대(再賃)한 경우
5. 임차인이 임차한 주택의 전부 또는 일부를 고의나 중대한 과실로 파손한 경우
6. 임차한 주택의 전부 또는 일부가 멸실되어 임대차의 목적을 달성하지 못할 경우
7. 주택의 전부 또는 대부분을 철거·재건축하기 위하여 점유를 회복할 필요가 있는 경우
7-1. 임대차계약 체결 당시 공사시기 및 소요기간 등 포함한 철거 또는 재건축 계획을 임차인에게 구체적으로 고지하고 그 계획에 따르는 경우
7-2. 건물이 노후하거나 훼손되는 등 안전사고의 우려가 있는 경우
7-3. 다른 법령에 따라 철거 또는 재건축이 이루어지는 경우
8. 임대인 또는 임대인의 직계존비속이 목적 주택에 실제 거주하려는 경우
(실거주자 성명:    임대인과의 관계: 본인/직계존속/직계비속)
9. 그 밖에 임차인이 임차인으로서의 의무를 현저히 위반하거나 임대차를 계속하기 어려운 중대한 사유가 있는 경우
위 계약갱신거절 사유를 보충설명하기 위한 구체적 사정
(기재란)
선택하신 사유를 소명할 수 있는 문서 등 별도의 자료가 있는 경우 해당 자료들을 본 통지서에 첨부하여 임차인에게 전달해 주시기 바랍니다.
작성일자    년    월    일 임대인 (서명 또는 날인)
